# **Proyecto #8: Clustering y visualización de curvas de luz de estrellas periódicas**
## **EL4106 - Inteligencia Computacional - Primavera 2022**
### **Parte 1 : Curvas de Luz y extracción de características**
##### **Profesor:** Pablo Estevez
##### **Auxiliar:** Juan Uturria 
##### **Ayudante:** Rafael de la Sotta Vargas
##### **Estudiantes:** Macarena Ríos - Melanie Peña

## 1) Librerías a importar para la experiencia

In [ ]:
import numpy as np
import pandas as pd
import random
%matplotlib inline
from google.colab import files
from google.colab import drive
drive.mount('/content/drive') #toda la data se fue cargada al drive personal para poder ejecutar

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Instalación de la libreria TurboFats para la extracción de características
!python -m pip install Cython
!python -m pip install -e git+https://git@github.com/alercebroker/turbo-fats#egg=turbofats
!python -m pip install -e git+https://git@github.com/alercebroker/mhps#egg=mhps
!python -m pip install -e git+https://git@github.com/alercebroker/P4J#egg=P4J
!python -m pip install pyarrow 
!python -m pip install -e git+https://git@github.com/alercebroker/lc_classifier#egg=lc_classifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining turbofats from git+https://****@github.com/alercebroker/turbo-fats#egg=turbofats
  Cloning https://****@github.com/alercebroker/turbo-fats to ./src/turbofats
  Running command git clone -q 'https://****@github.com/alercebroker/turbo-fats' /content/src/turbofats
  Running setup.py develop for turbofats
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining mhps from git+https://****@github.com/alercebroker/mhps#egg=mhps
  Cloning https://****@github.com/alercebroker/mhps to ./src/mhps
  Running command git clone -q 'https://****@github.com/alercebroker/mhps' /content/src/mhps
  Running setup.py develop for mhps
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining P4J from gi

Una vez ejecutado el bloque anterior, reiniciar el entorno de ejecución para poder importar las siguientes funciones relevantes para el desarrollo del proyecto.

In [ ]:
#características de la libreria a utilizar 
from lc_classifier.features import MHPSExtractor, PeriodExtractor, GPDRWExtractor
from lc_classifier.features import FoldedKimExtractor
from lc_classifier.features import HarmonicsExtractor, IQRExtractor
from lc_classifier.features import PowerRateExtractor
from lc_classifier.features import TurboFatsFeatureExtractor
from lc_classifier.features import FeatureExtractorComposer

## 2) Cargar la data a utilizar 

In [ ]:
labels =  pd.read_csv("/content/drive/MyDrive/labels_set.csv")
alerts =  pd.read_csv("/content/drive/MyDrive/alert_detections_V2.csv",iterator=True,chunksize=100000) #debido a que la data de alerts es demasiado grande, se debe recorrer
                                                                                                       #en pedazos

In [ ]:
labels #visualización de todo el inventario de estrellas que existen

,oid,classALeRCE,ra,dec,source,id_source
0,ZTF19aapcxhy,AGN,154.202129,18.723076,Oh2015,5.877420127343739e+17
1,ZTF18abtmwvo,AGN,46.074050,0.474212,Oh2015,5.880155098263717e+17
2,ZTF18acvgdfy,AGN,134.407409,5.472596,Oh2015,5.877327033915148e+17
3,ZTF19aabvjsi,AGN,132.353298,14.067266,Oh2015,5.877420137987442e+17
4,ZTF18aadyxlg,AGN,125.577004,33.091120,Oh2015,5.880133827239608e+17
...,...,...,...,...,...,...
123491,ZTF17aaanrhv,YSO,100.188180,9.478992,Simbad_variables,EM* LkHA 364
123492,ZTF18aabgmld,YSO,100.248642,9.478838,Simbad_variables,V* LV Mon
123493,ZTF17aaarpje,YSO,84.062814,-6.293581,Simbad_variables,V* BB Ori
123494,ZTF18actawih,YSO,83.880279,-5.669713,Simbad_variables,V* V1558 Ori


In [ ]:
labels.groupby(by="classALeRCE").count()["oid"].to_dict() #desbalance de clases, 15 clases dentro del inventario

{'AGN': 4667,
 'Blazar': 1267,
 'CEP': 618,
 'CV/Nova': 871,
 'DSCT': 732,
 'E': 37901,
 'LPV': 14076,
 'Periodic-Other': 1256,
 'QSO': 26168,
 'RRL': 32482,
 'SLSN': 24,
 'SNII': 328,
 'SNIa': 1272,
 'SNIbc': 94,
 'YSO': 1740}

## 3) Obtención de las estrellas periódicas

In [ ]:
#Debido a que el proyecto corresponde especificamente a estrellas periódicas, procedemos eliminando todas las estrellas que no correspondan de la data de labels
#creando una función auxiliar que recorra la data de labels y dropee aquellas estrellas no necesarias
#Clases de estrellas periódicas : Long Period Variable (LPV), Cepheid (CEP), RR Lyrae (RRL), δ Scuti (DSCT), Eclipsing Binary (E) y Periodic-Other
def periodic(data):
  nonperiodic = ['AGN', 'Blazar', 'CV/Nova', 'QSO', 'SLSN', 'SNII','SNIa','SNIbc','YSO'] #estrellas no periodicas
  for i in range(len(nonperiodic)):
    index1 = data[data["classALeRCE"] == nonperiodic[i]].index 
    data.drop(index1, inplace=True)
  return labels

In [ ]:
periodic(labels)

,oid,classALeRCE,ra,dec,source,id_source
6805,ZTF18abwwdsc,CEP,326.755845,-8.060673,CRTSnorth,1007116010622.0
6806,ZTF18aaakigd,CEP,65.440016,34.069809,CRTSnorth,1135020009815.0
6807,ZTF18abwwdxw,CEP,324.865973,-17.296145,CRTSnorth,1018112055304.0
6808,ZTF19aaocniv,CEP,248.580619,-16.015739,CRTSnorth,1015086048064.0
6809,ZTF17aacemqz,CEP,117.977671,27.561363,CRTSnorth,1126038065052.0
...,...,...,...,...,...,...
120033,ZTF18abwwdwe,Periodic-Other,330.079004,-10.046703,CRTSnorth,1009117031057.0
120034,ZTF19aceilmk,Periodic-Other,312.373578,1.105685,CRTSnorth,1101112046278.0
120035,ZTF18abtyrpc,Periodic-Other,351.078025,31.870857,CRTSnorth,1132108024048.0
120036,ZTF18actuvrp,Periodic-Other,203.937280,-8.807032,CRTSnorth,1009072054924.0


In [ ]:
labels.groupby(by="classALeRCE").count()["oid"].to_dict() 

{'CEP': 618,
 'DSCT': 732,
 'E': 37901,
 'LPV': 14076,
 'Periodic-Other': 1256,
 'RRL': 32482}

In [ ]:
#este bloque sirve para guardar la data de las estrellas periodicas, descomentar en caso de que se desee obtener el csv correspondiente
#labels.to_csv('periodicas.csv') 
#files.download('periodicas.csv')

## 4) Sampling de estrellas y Curvas de Luz

Como se puede verificar en la sección 3) es evidente notar para obtener un balanceo entre las clases hay que tener en consideración que la cantidad de estrellas que pertenecen a la clase CEP son 618. Por ende, no se le puede pedir una cantidad mayor a esta clase. Por esta razón se van a utilizar un sample de 600 estrellas para cada una de las clases, sin embargo, para la clase CEP se van a utilizar las 618 estrellas en su totalidad.

Para poder obtener un sample de cada una de las clases estrellas se decidió como primer paso obtener la data POR SEPARADO de cada una de las clases, con esto se quiere decir que se creo un dataset para la clase CEP, uno para la clase RRL, y lo mismo para las otras clases restantes. Para esto se utilizo la siguiente función auxiliar:

In [ ]:
def single_class(data):
  nointerest = ['CEP','RRL','DSCT','E','Periodic-Other'] #obtener data LPV
  #nointerest = ['LPV','RRL','DSCT','E','Periodic-Other'] #Obtener data CEP
  #nointerest = ['LPV','CEP','DSCT','E','Periodic-Other'] #obtener data RRL
  #nointerest = ['LPV','CEP','RRL','E','Periodic-Other'] #obtener data DSCT
  #nointerest = ['LPV','CEP','RRL','DSCT','Periodic-Other'] #obtener data E
  #nointerest = ['LPV','CEP','RRL','DSCT','E'] #obtener data Periodic-Other
  for i in range(len(nointerest)):
    index2 = data[data["classALeRCE"] == nointerest[i]].index 
    data.drop(index2, inplace=True)
  return data

In [ ]:
single_class(labels)

,oid,classALeRCE,ra,dec,source,id_source
46056,ZTF18abucchs,LPV,315.078308,-6.115294,CRTSnorth,1007112078870.0
46057,ZTF18aabhkdy,LPV,106.498180,52.477287,CRTSnorth,1152024060987.0
46058,ZTF18aagvrdt,LPV,247.282715,34.229506,CRTSnorth,1135073011544.0
46059,ZTF18aaiylnl,LPV,251.918113,56.967817,CRTSnorth,1157050015163.0
46060,ZTF18aabebwy,LPV,187.742089,12.308543,CRTSnorth,1112066024303.0
...,...,...,...,...,...,...
60127,ZTF18aaxymzx,LPV,295.919157,22.724404,GAIADR2VS,2.0199127469765652e+18
60128,ZTF18abikbvc,LPV,286.720222,24.733440,GAIADR2VS,4.533629080944307e+18
60129,ZTF19aamtoyb,LPV,99.257590,-15.567786,GAIADR2VS,2.9501070744892713e+18
60130,ZTF18abnubuy,LPV,292.053754,38.418277,GAIADR2VS,2.0526959231045647e+18


Como se puede verificar se obtuvo de manera exitosa la data total correspondiente a la data de la clase LPV. Para poder obtener la data de cada una de las clases se debe ejecutar los bloques de cargar la data labels (sección 2), la obtención de la data para las estrellas periódicas (sección 3) y finalmente descomentar la línea de código dentro de la función single_class correspondiente para obtener cada una de las datas deseadas. Se procede a guardar cada una de estas datas y se guardan en las siguientes variables. 

In [ ]:
LPV = pd.read_csv("/content/drive/MyDrive/Proyecto8/DataFrames /LPV.csv")
CEP = pd.read_csv("/content/drive/MyDrive/Proyecto8/DataFrames /CEP.csv")
RRL = pd.read_csv("/content/drive/MyDrive/Proyecto8/DataFrames /RRL.csv")
DSCT = pd.read_csv("/content/drive/MyDrive/Proyecto8/DataFrames /DSCT.csv")
E = pd.read_csv("/content/drive/MyDrive/Proyecto8/DataFrames /E.csv")
PeriodicOther = pd.read_csv("/content/drive/MyDrive/Proyecto8/DataFrames /PeriodicOther.csv")

Para los siguientes pasos a seguir, se va a utilizar solo una de clases para la demostración de lo que se debe utilizar con todas las siguientes, de todas formas se entregan las líneas de código comentadas para realizar el proceso con todas las clases. La clase a utilizar como ejemplo corresponde a la clase LPV.

In [ ]:
#Como primer paso, se deben obtener todos los id de las estrellas de cada una de las clases

oids = LPV["oid"]
#oids = RRL["oid"]
#oids = DSCT["oid"]
#oids = E["oid"]
#oids = PeriodicOther["oid"]
#no existe este sección para la clase CEP, debido a que no se realiza un sampling para esta clase, se utilizan
#las 618 estrellas en su totalidad


#Sampling de 600 estrellas para cada clase, estos samples cambian cada vez que se ejecuta debido a que se
#utiliza la librería random
oid_array = oids.to_numpy()

def tolist(array):
  list=[]
  for i in range(len(array)): list.append(array[i])
  return list

oid_list = tolist(oid_array) 
oid_sample = random.sample(oid_list, 600)
oid_sample

['ZTF19aargivj',
 'ZTF18abeojzu',
 'ZTF18ablqphs',
 'ZTF18abdihyu',
 'ZTF18abecfha',
 'ZTF18abmozjd',
 'ZTF19aazrjzs',
 'ZTF18abegcjn',
 'ZTF18abusfrv',
 'ZTF18aahdrhl',
 'ZTF18aazvdop',
 'ZTF18abmagoo',
 'ZTF18abjwmnk',
 'ZTF18abeymgq',
 'ZTF18aazvxtd',
 'ZTF18abmewoh',
 'ZTF18abfxgjy',
 'ZTF18ablqldx',
 'ZTF18abloqju',
 'ZTF18abeelyq',
 'ZTF18abdhxbc',
 'ZTF18abfyvpj',
 'ZTF17aaatizh',
 'ZTF18abcwyuj',
 'ZTF18abnouyz',
 'ZTF18aazmpvx',
 'ZTF17aaadzrt',
 'ZTF18ablpvwt',
 'ZTF18aavyvgn',
 'ZTF18abcjupf',
 'ZTF18abgjkab',
 'ZTF18acakuxw',
 'ZTF18abnbnxt',
 'ZTF18abizqrv',
 'ZTF17aaapvrg',
 'ZTF18aazfeah',
 'ZTF18absmddt',
 'ZTF18aasnizu',
 'ZTF18aavfequ',
 'ZTF18aawsrvo',
 'ZTF18aayfbbb',
 'ZTF18abdlbtu',
 'ZTF18aayeigr',
 'ZTF18abmvwml',
 'ZTF18abnjyxg',
 'ZTF18aaxahvh',
 'ZTF18abmnyky',
 'ZTF18abcurws',
 'ZTF19aaxtwjs',
 'ZTF19aaeslxf',
 'ZTF17aaawgkz',
 'ZTF18abslvkv',
 'ZTF18abgfmix',
 'ZTF18abcbspm',
 'ZTF18ablvhgn',
 'ZTF18abvcjao',
 'ZTF18abgzyiw',
 'ZTF18abncaon',
 'ZTF18aasnnjn

Una vez definido el sample para cada una de las clases, se procede a obtener las curvas de luz correspondiente a estas estrellas recorriendo la data de alerts. 

In [ ]:
def light_curve(array_oid,alerts):
  df_array = []
  for chunk in alerts:
      aux_df = chunk[chunk['oid'].isin(array_oid)]
      df_array.append(aux_df)
  df = pd.concat(df_array)
  df.dropna(inplace=True)
  df.reset_index(inplace=True,drop=True)
  return df

In [ ]:
df = light_curve(oid_sample,alerts)
df

,oid,candid,fid,mjd,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext
0,ZTF18aaxdoeg,518400060115010004,1,58272.400069,16.299545,100.000000,0.008553
1,ZTF18aaxdoeg,715119490115015007,1,58469.119491,15.210057,100.000000,0.016599
2,ZTF18abfitme,955208613315010005,2,58709.208611,15.743909,100.000000,0.020520
3,ZTF18aaxjjot,950289083115010012,2,58704.289086,14.963398,100.000000,0.011725
4,ZTF18aaxjjot,1138567223115010013,1,58892.567222,17.243220,0.041481,0.045674
...,...,...,...,...,...,...,...
94714,ZTF18aaupzra,833509510015010023,2,58587.509514,14.864432,100.000000,0.030290
94715,ZTF18aaupzra,891421891315010033,1,58645.421898,17.985100,0.047886,0.060525
94716,ZTF18abyohjt,1037299532315010003,1,58791.299537,18.066425,0.126611,0.156899
94717,ZTF18abypodv,1120204820815010006,1,58874.204826,20.365503,2.358758,3.003142


## 5) Extracción de características

A partir de las curvas de luz es que pueden extraer las características que se desean observar para cada una de las estrellas. Estas características se deben extraer de las bandas 'r' y 'g', por lo que se deben asignar estas bandas a la data anterior.

In [ ]:
def preprocessing(df):
  df_array = []
  for oid in df['oid'].unique():
    df_aux = df[df['oid']==oid]
    min_time =  df_aux['mjd'].min()
    df_aux['mjd'] = df_aux['mjd'] - min_time
    df_array.append(df_aux)
  df = pd.concat(df_array)
  df.replace({'fid': {1: 'g', 2: 'r'}},inplace=True)
  df = df[['oid','mjd','magpsf_corr','sigmapsf_corr_ext', 'fid']]
  df.set_index('oid', inplace=True)
  df.rename(columns={'mjd': 'time', 'magpsf_corr': 'magnitude', 'sigmapsf_corr_ext': 'error', 'fid': 'band'}, inplace=True)
  return df

In [ ]:
df_prep = preprocessing(df)
df_prep

 /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,time,magnitude,error,band
oid,,,,
ZTF18aaxdoeg,5.934896,16.299545,0.008553,g
ZTF18aaxdoeg,202.654317,15.210057,0.016599,g
ZTF18aaxdoeg,46.894051,16.919165,0.130291,g
ZTF18aaxdoeg,405.874259,16.964485,0.138726,g
ZTF18aaxdoeg,738.013102,16.717918,0.097689,g
...,...,...,...,...
ZTF19aawzasm,45.022535,15.503465,0.012745,r
ZTF19aawzasm,7.062384,15.790150,0.013578,r
ZTF19aawzasm,14.020752,15.755711,0.007959,r


Una vez asignadas estas bandas, es que se puede utilizar la librería TurboFats instalada en un inicio para la extracción de las características.

In [ ]:
def features(df):
  df_array = []
  for oid in df.index.unique():
    df_aux = df[df.index==oid]
    bands = ['r','g']
    feature_extractor = FeatureExtractorComposer(
        [
            MHPSExtractor(bands),
            PeriodExtractor(bands),
            GPDRWExtractor(bands),
            FoldedKimExtractor(bands),
            HarmonicsExtractor(bands),
            IQRExtractor(bands),
            PowerRateExtractor(bands),
            TurboFatsFeatureExtractor(bands)
        ]
    )
    features = feature_extractor.compute_features(df_aux)
    df_array.append(features)
  return pd.concat(df_array)

In [ ]:
df_features = features(df_prep)
df_features

 /content/src/turbofats/turbofats/FeatureFunctionLib.py:687: RuntimeWarning:divide by zero encountered in true_divide
 /content/src/turbofats/turbofats/FeatureFunctionLib.py:693: RuntimeWarning:invalid value encountered in double_scalars
 /content/src/turbofats/turbofats/FeatureFunctionLib.py:437: RuntimeWarning:divide by zero encountered in true_divide
 /content/src/turbofats/turbofats/FeatureFunctionLib.py:687: RuntimeWarning:divide by zero encountered in true_divide
 /content/src/turbofats/turbofats/FeatureFunctionLib.py:693: RuntimeWarning:invalid value encountered in double_scalars
 /content/src/turbofats/turbofats/FeatureFunctionLib.py:437: RuntimeWarning:divide by zero encountered in true_divide
 /content/src/turbofats/turbofats/FeatureFunctionLib.py:690: RuntimeWarning:invalid value encountered in multiply
 /content/src/turbofats/turbofats/FeatureFunctionLib.py:437: RuntimeWarning:invalid value encountered in true_divide
 /content/src/turbofats/turbofats/FeatureSpace.py:57: Ran

,MHPS_ratio_r,MHPS_low_r,MHPS_high_r,MHPS_non_zero_r,MHPS_PN_flag_r,MHPS_ratio_g,MHPS_low_g,MHPS_high_g,MHPS_non_zero_g,MHPS_PN_flag_g,...,Skew_g,SmallKurtosis_g,Std_g,StetsonK_g,Pvar_g,ExcessVar_g,SF_ML_amplitude_g,SF_ML_gamma_g,IAR_phi_g,LinearTrend_g
oid,,,,,,,,,,,,,,,,,,,,,
ZTF18aaxdoeg,NaN,NaN,NaN,NaN,NaN,858.060608,14.731952,0.017169,210.0,0.0,...,-1.406576,1.557065,0.666209,0.624368,1.0,0.001593,6.311979,1.236532,0.996571,-0.000680
ZTF18abfitme,679.543152,0.422975,0.000622,103.0,0.0,17.493587,0.056117,0.003208,94.0,1.0,...,-2.522532,14.533493,0.137235,0.503983,1.0,0.000029,2.346344,1.339182,0.986195,0.000174
ZTF18aaxjjot,291.982910,0.609018,0.002086,110.0,0.0,74.698135,0.085363,0.001143,115.0,1.0,...,-0.680564,-0.972946,0.261611,0.945855,1.0,0.000225,0.282047,0.598197,0.997247,0.001137
ZTF19abyrccj,822.149719,0.431830,0.000525,114.0,0.0,2520.500732,0.935664,0.000371,149.0,0.0,...,-2.059650,5.910410,0.119814,0.638703,1.0,0.000060,0.797880,0.780156,0.985235,-0.001388
ZTF19aadunfm,298.188385,0.351813,0.001180,23.0,0.0,5486.484863,0.917905,0.000167,5.0,0.0,...,-0.216204,-0.661194,0.207553,0.951489,1.0,0.000167,0.548506,0.418724,0.982293,-0.000838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF19aapcnux,NaN,NaN,NaN,NaN,NaN,237.917679,1.745530,0.007337,13.0,0.0,...,0.044173,-1.737380,1.077857,0.870273,1.0,0.005321,4.788002,0.866140,0.997429,0.007150
ZTF18abpfgtx,218.907349,0.416913,0.001905,9.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF19aaxljax,0.846344,0.000281,0.000332,6.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.033698,-1.832066,0.088178,0.934817,1.0,0.000039,0.428964,1.157633,0.997554,0.000578


Finalmente se obtuvieron las características de toda la clase LPV como se especificó en un inicio. Descomentando las líneas de código corrrespondientes en la sección 4) se obtienen los dataframes correspondientes a las características de cada una de las clases. Esta data se procede a guardar para poder utilizar en la segunda parte del proyecto.

In [ ]:
#Código de ejemplo para guardar la data de las características
#df_features.to_csv('LPV_features.csv') 
#files.download('LPV_features.csv')